# KBMOD Demo

In [ ]:
import os
import numpy as np

from pathlib import Path

from kbmod.run_search import *
from kbmod.search import *

# Setup file paths

There are at least two file paths you need to setup in order to run kbmod:
1. The im_filepath provides a path to the input images.
1. The res_filepath provides a path to the directory where the output results will be stored.

A time and psf file can optionally be specified.

If you already have data files, you can use those. Below we use the data in `data/demo`. You can also create your own fake data using `fake_data_creator.py`.

In [ ]:
# Define the path for the data.
im_filepath = "../data/demo"
print(os.listdir(im_filepath))

In [ ]:
res_filepath = "./fake_results"
if not Path(res_filepath).is_dir():
    print(f"Directory {res_filepath} does not exist. Creating.")
    os.mkdir(res_filepath)

# Run KBMOD

The standard approach to running KBMOD is to perform a grid search over all starting pixels and a grid of velocities. The velocities are defined by steps in velocity space (in pixels per day) and angles. Let’s do a grid search with:
* 21 different velocity steps from 0 pixels per day and 20 pixels per day
* 11 different angles from 0.5 below the default angle (computed based on the ecliptic) to 0.5 above.

KBMOD needs a series of configuration parameters to specify all the information about the search. In this notebook we explicitly provide the configuration parameters as a dictionary so users can see what is being specified. However most users will want to use the ``SearchConfiguration`` class. A ``SearchConfiguration`` object uses reasonable defaults when created:

```
config = SearchConfiguration()
```

Users can then override values one at a time or by passing a dictionary:

```
d = {"im_filepath": "Here", "encode_num_bytes": 2}
config.set_multiple(d)
```

More importantly ``SearchConfiguration`` can read from or written to a YAML file:

```
config = SearchConfiguration.from_file(file_path)
```

This allows users to define a per-task configuration and version control it.

Most of the parameters you will not need to change. They are included to provide fine grained options for control.

In [ ]:
results_suffix = "DEMO"

# The demo data has an object moving at x_v=10 px/day
# and y_v = 0 px/day. So we search velocities [0, 20].
v_min = 0
v_max = 20
v_steps = 21
v_arr = [v_min, v_max, v_steps]

# and angles [-0.5, 0.5]
ang_below = 0.5
ang_above = 0.5
ang_steps = 11
ang_arr = [ang_below, ang_above, ang_steps]

input_parameters = {
    # Input parameters
    "im_filepath": im_filepath,
    # Grid search parameters
    "v_arr": v_arr,
    "ang_arr": ang_arr,
    # Output parameters
    "res_filepath": res_filepath,
    "output_suffix": results_suffix,
    "result_filename": "./fake_results/results.ecsv",
    # Masking bad pixels
    "do_mask": True,  # <-- Apply the masks specified in the FITS files.
    "mask_num_images": 10,
    # Basic filtering (always applied)
    "num_obs": 7,  # <-- Filter anything with fewer than 7 observations
    "lh_level": 10.0,  # <-- Filter anything with a likelihood < 10.0
    # SigmaG clipping parameters
    "sigmaG_lims": [15, 60],  # <-- Clipping parameters (lower and upper percentile)
    "gpu_filter": True,  # <-- Apply clipping and filtering on the GPU
    "clip_negative": True,
    # Override the ecliptic angle for the demo data since we
    # know the true angle in pixel space.
    "average_angle": 0.0,
}
config = SearchConfiguration.from_dict(input_parameters)

To see the full list of parameters used and their values you can just print the configuration.

In [ ]:
# print(config)

KBMOD uses Python's logging library for output during the run. If you are interested in running with debug output (verbose mode), you can set the level of Python's logger to ``WARNING`` (for warning messages only), ``INFO`` (for moderate output), or ``DEBUG`` (for comprehensive output). By default logging is set to warning level.

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

# Or for a LOT of detail
# logging.basicConfig(level=logging.DEBUG)

Once we have defined the search parameters, we can create a ``SearchRunner`` and use one of the run_search functions. In this case we use ``run_search_from_config`` which uses the config to search for the input files.

In [ ]:
rs = SearchRunner()
results = rs.run_search_from_config(config)

We then check that results were written to an output directory. The configuration parameters above specify that KBMOD should write three types of output files:

1. A combined serialized ``Results`` saved as a .ecsv file (``"result_filename": "./fake_results/results.ecsv")``.
2. (Legacy format) A series of individual output files (``"res_filepath": res_filepath``). Currently this is just the results file (trajectory information) and a copy of the final configuration used. Recent versions of KBMOD has removed older files, such as the psi curves or phi curves, that were not being used. However we can easily add files that would be useful.

Users can shut off these outputs but passing ``None`` to the configuration options.

In [ ]:
if os.path.exists(res_filepath):
    files = os.listdir(res_filepath)
    print(files)

# Analyzing Results

The run function we used returns a `Results` object containing the individual results of the run. We can perform basic actions on this data structure such as sorting it, extracting individual results, or performing additional filtering.

In [ ]:
print(f"Search found {len(results)} results.")
print(results)

We can also plot the different curves for the result.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 1)
axs[0].plot(results["psi_curve"][0])
axs[0].set_title("Psi")

axs[1].plot(results["phi_curve"][0])
axs[1].set_title("Psi")

For additional analysis steps, including manual filtering and exploration, please refer to the `kbmod_analysis_demo` notebook which uses the data generated by this notebook.